Calculation of the spatial relations based on a dataset.

In [1]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import datasets

In [2]:
ds_2014 = datasets.load('2014_segments')
ds_xy = ds_2014[['x', 'y']]
datasets.save(ds_xy, '2014_xy', overwrite=True)

In [3]:
def object_dist(ds, obj, i):
    return ((ds.to_array(name=f'objdist_{i}').T - obj)**2).sum('variable')**0.5
def min_object_dist(ds):
    arrays = [object_dist(ds, obj, i) for i,obj in enumerate(ds.objects)]
    return xr.merge(arrays).to_array('dist_object', name='dist_object').min('dist_object')

In [4]:
def line_dist(ds, line, i):
    l2 = sum((line[0]-line[1])**2)
    if l2 == 0:
        return object_dist(ds, line[0], i)
    da = ds.to_array(name=f'linedist_{i}').T
    t = np.dot(da-line[0], line[1] - line[0]) / l2
    projection = (line[0] + np.outer(t, (line[1] - line[0]))).reshape(da.shape)
    return ((da-projection)**2).sum('variable')**0.5
def min_line_dist(ds):
    boundary = np.concatenate((ds.boundary[-1:],ds.boundary))
    lines = [boundary[i:i+2] for i in range(len(ds.boundary))]
    arrays = [line_dist(ds, line, i) for i,line in enumerate(lines)]
    return xr.merge(arrays).to_array('dist_boundary', name='dist_boundary').min('dist_boundary')

In [5]:
def absolute_position(ds):
    return ds.to_array('component', name='pos_abs').T
def relative_position(ds):
    return ds.diff('time').to_array('component', name='pos_rel').T

In [15]:
def spatial_relations(ds):
    pos_abs = absolute_position(ds)
    pos_rel = relative_position(ds)
    dist_object = min_object_dist(ds)
    dist_boundary = min_line_dist(ds)
    r = xr.merge([pos_abs, pos_rel, dist_object, dist_boundary])
    return r

In [16]:
ds_sr = spatial_relations(ds_xy)
datasets.save(ds_sr, '2014_spatrel', overwrite=True)

In [17]:
ds_sr

<xarray.Dataset>
Dimensions:        (component: 2, segment: 240, time: 15000)
Coordinates:
  * time           (time) timedelta64[ns] 00:00:00 00:00:00.040000 ...
    rat            (segment) object '110' '110' '110' '110' '110' '110' ...
    trial          (segment) object '5' '5' '5' '17' '17' '17' '29' '29' ...
    treatment      (segment) object 's' 's' 's' 's' 's' 's' 's' 's' 's' 's' ...
    part           (segment) int64 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 ...
    session        (segment) object 'injection1' 'injection1' 'injection1' ...
  * component      (component) <U1 'x' 'y'
Dimensions without coordinates: segment
Data variables:
    pos_abs        (segment, time, component) float64 -3.408 2.523 -3.288 ...
    pos_rel        (segment, time, component) float64 nan nan 0.1204 0.2022 ...
    dist_object    (segment, time) float64 42.22 42.0 41.73 41.4 41.05 40.73 ...
    dist_boundary  (segment, time) float64 71.01 71.12 71.26 71.5 71.77 ...